In [2]:
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

In [3]:
#1. build a random forest classifier with 'longer' text
import pandas as pd
df_train = pd.read_csv("/Users/chengyiyang/Downloads/tweets_mis-filtrd4.csv", engine='python')
df_train

FileNotFoundError: [Errno 2] No such file or directory: '/Users/chengyiyang/Downloads/tweets_mis-filtrd4.csv'

In [40]:
from sklearn.model_selection import train_test_split

#sub_df_train = df_train[['Tokens','Label']].copy()
#label_map = dict(zip(df_train['Label'].unique().tolist(), range(6)))
#sub_df_train['Flipped_label'] = sub_df_train['Label'].apply(lambda x: label_map[x])
xs = sub_df_train['Tokens'].values
ys = sub_df_train['Label'].values
train_x, test_x, train_y, test_y = train_test_split(xs, ys, random_state=42, test_size=0.15)

In [41]:
print(test_x[111], test_y[111]
     
     )

genius 0


In [7]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer

from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.feature_extraction.text import TfidfTransformer 

vectorizer=CountVectorizer(max_features=100)


clf = RandomForestClassifier()
c = make_pipeline(vectorizer, clf)

c.fit(train_x, train_y)

Pipeline(steps=[('countvectorizer', CountVectorizer(max_features=100)),
                ('randomforestclassifier', RandomForestClassifier())])

In [8]:
vect_2 = CountVectorizer(stop_words='english', max_features=100)
c2 = make_pipeline(vect_2, clf)
c2.fit(train_x, train_y)

Pipeline(steps=[('countvectorizer',
                 CountVectorizer(max_features=100, stop_words='english')),
                ('randomforestclassifier', RandomForestClassifier())])

In [9]:
print (c.predict_proba(test_x[0:10]), c2.predict_proba(test_x[0:10]))

[[0.82837449 0.17162551]
 [1.         0.        ]
 [0.87074956 0.12925044]
 [0.8629043  0.1370957 ]
 [0.93216047 0.06783953]
 [0.18090164 0.81909836]
 [0.94181232 0.05818768]
 [1.         0.        ]
 [0.8629043  0.1370957 ]
 [0.97747127 0.02252873]] [[0.11282276 0.88717724]
 [0.8629043  0.1370957 ]
 [0.71350148 0.28649852]
 [0.8629043  0.1370957 ]
 [0.93216047 0.06783953]
 [0.70739285 0.29260715]
 [0.68884824 0.31115176]
 [0.8629043  0.1370957 ]
 [0.8629043  0.1370957 ]
 [1.         0.        ]]


In [17]:
from lime.lime_text import LimeTextExplainer
class_names = ['Non-Toxic', 'Toxic']
explainer = LimeTextExplainer(class_names=class_names)

In [39]:
idx = 21
exp = explainer.explain_instance(test_x[idx], c2.predict_proba, num_features=6)
print('Document id: %d' % idx)
print('Probability(non-toxic) =', c2.predict_proba([test_x[idx]])[0,1], c.predict_proba([test_x[idx]])[0,1])
print('True class: %s' % class_names[test_y[idx]])
print('Tweet: %s' % test_x[idx])

Document id: 21
Probability(non-toxic) = 0.1370957014586832 0.8408862803862802
True class: Non-Toxic
Tweet: kandre are such bad sports


In [24]:
print(test_x[500
            ])

yup clemson uni


In [29]:
exp_2 = explainer.explain_instance(test_x[idx], c.predict_proba, num_features=6)
print(exp.as_list(), exp_2.as_list())


[('hate', 0.15131205521371688), ('shadowrun', -0.05346704258417674)] [('shadowrun', 1.1859480308508826e-30), ('hate', 1.1745284392340069e-30)]
